In [20]:
import exiftool
from astropy.time import Time
import astropy.units as u
raw_path = '291A2094.CR3'
with exiftool.ExifToolHelper() as et:
    exif = et.get_metadata(raw_path)[0]
    time = exif['EXIF:DateTimeOriginal']
    offset = exif['EXIF:OffsetTime']
time = time.replace(':','-',2)
time = Time(time)-int(offset[0:3])*u.hour

In [31]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, angular_separation, position_angle, offset_by
bj = EarthLocation(lon=116,lat=40,height = 200)

In [34]:
def alt2pres(altitude):
    # https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/atmosphere.html
    press = 100 * ((44331.514 - altitude) / 11880.516) ** (1 / 0.1902632)

    return press/100*u.hPa

f1 = AltAz(obstime=Time.now(), location=bj, pressure = alt2pres(bj.height.value),temperature=0*u.deg_C,relative_humidity=0.5,obswl=550*u.nm)
f2 = AltAz(obstime=Time.now(), location=bj)
s = SkyCoord(ra=0,dec=45,frame='icrs', unit='deg')
s.transform_to(f1).alt-s.transform_to(f2).alt

<Angle 0.12092566 deg>

In [37]:
type(s.transform_to(f1))

astropy.coordinates.sky_coordinate.SkyCoord

In [40]:
dir(s.transform_to(f1))

['T',
 '_APPLICABLE_FUNCTIONS',
 '_METHOD_FUNCTIONS',
 '__abstractmethods__',
 '__array_function__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_apply',
 '_extra_frameattr_names',
 '_is_name',
 '_sky_coord_frame',
 'alt',
 'altaz',
 'apply_space_motion',
 'az',
 'barycentricmeanecliptic',
 'barycentrictrueecliptic',
 'cache',
 'cartesian',
 'cirs',
 'contained_by',
 'copy',
 'custombarycentricecliptic',
 'cylindrical',
 'data',
 'default_differential',
 'default_representation',
 'diagonal',
 'differential_type',
 'directional_offset_by',
 'distanc

In [22]:
EarthLocation

Unit("pa")

In [10]:
t = Time(time)

ValueError: Input values did not match any of the formats where the format keyword is optional: {'datetime': TypeError('Input values for datetime class must be datetime objects'), 'ymdhms': ValueError('input must be dict or table-like'), 'iso': ValueError('Time 2024:03:07 21:11:25 does not match iso format'), 'isot': ValueError('Time 2024:03:07 21:11:25 does not match isot format'), 'yday': ValueError('Time 2024:03:07 21:11:25 does not match yday format'), 'datetime64': TypeError('Input values for datetime64 class must be datetime64 objects'), 'fits': ValueError('Time 2024:03:07 21:11:25 does not match fits format'), 'byear_str': ValueError('Time 2024:03:07 21:11:25 does not match byear_str format'), 'jyear_str': ValueError('Time 2024:03:07 21:11:25 does not match jyear_str format'), 'astropy_time': TypeError('Input values for astropy_time class must all be the same astropy Time type.')}

In [11]:
with exiftool.ExifToolHelper() as et:
    exif = et.get_metadata(raw_path)[0]
exif

{'SourceFile': '291A2094.CR3',
 'ExifTool:ExifToolVersion': 12.78,
 'File:FileName': '291A2094.CR3',
 'File:Directory': '.',
 'File:FileSize': 30253832,
 'File:FileModifyDate': '2024:03:07 21:11:36+08:00',
 'File:FileAccessDate': '2024:06:03 15:03:42+08:00',
 'File:FileCreateDate': '2024:05:24 15:48:56+08:00',
 'File:FilePermissions': 100666,
 'File:FileType': 'CR3',
 'File:FileTypeExtension': 'CR3',
 'File:MIMEType': 'image/x-canon-cr3',
 'File:ExifByteOrder': 'II',
 'QuickTime:MajorBrand': 'crx ',
 'QuickTime:MinorVersion': '0.0.1',
 'QuickTime:CompatibleBrands': ['crx ', 'isom'],
 'QuickTime:MovieHeaderVersion': 0,
 'QuickTime:CreateDate': '2024:03:07 21:11:25+08:00',
 'QuickTime:ModifyDate': '2024:03:07 21:11:25+08:00',
 'QuickTime:TimeScale': 1,
 'QuickTime:Duration': 1,
 'QuickTime:PreferredRate': 1,
 'QuickTime:PreferredVolume': 1,
 'QuickTime:PreviewTime': 0,
 'QuickTime:PreviewDuration': 0,
 'QuickTime:PosterTime': 0,
 'QuickTime:SelectionTime': 0,
 'QuickTime:SelectionDuratio